# Query Meta data in database Groups [v1.1]

In [1]:
# imports
from astropy import units as u
from astropy.coordinates import SkyCoord

import specdb
from specdb.specdb import SpecDB
from specdb import specdb as spdb_spdb
from specdb.cat_utils import flags_to_groups

## Setup

In [2]:
db_file = specdb.__path__[0]+'/tests/files/IGMspec_DB_v02_debug.hdf5'
reload(spdb_spdb)
sdb = spdb_spdb.SpecDB(db_file=db_file)

Database is igmspec
Created on 2017-Jan-07


## Check one of the meta tables

In [3]:
ggg_meta = sdb['GGG'].meta
ggg_meta[0:4]

SDSSJ,RA_GROUP,DEC_GROUP,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAG,JMAGERR,HMAG,HMAGERR,KMAG,KMAGERR,MASSDEL,MASSFLG,MIMAG,DGMI,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,PLATE,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag,zem_GROUP,sig_zem,flag_zem,EPOCH,STYPE,IGM_ID,SPEC_FILE,TELESCOPE,INSTR,DATE-OBS,NPIX,WV_MIN,WV_MAX,R,GROUP_ID,DISPERSER
str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str31,str31,str31,float64,float64,float64,float64,float64,float64,float64,float64,str14,float64,float32,int32,int32,int32,int32,float32,float32,int32,float32,float32,int32,float32,float32,str3,float64,str3,int64,str27,str12,str6,str10,int64,float64,float64,float64,int64,str4
001115.23+144601.8,2.8135,14.7672,4.9672,24.087,0.827,22.081,0.074,19.617,0.021,18.277,0.014,18.101,0.025,0.272,20.643,-1.0,0.0,0.0,-9.0,0.0,0.0,17.547,0.342,16.76,0.257,15.796,0.244,0.0,26,-29.191,-0.922,0,1,1,1,8388609,0,1,0,0,1,0,0,2566,52170,52251,751,39,40,3,327,266,8388609,0,1,0,0,1,0,0,24.385,0.941,22.071,0.073,19.642,0.018,18.271,0.013,18.14,0.023,587730773889974538,211612124009463808,SDSS J001115.23+144601.8,1.934192,2.409056,1.304912,0.143904,2.242192,2.374056,1.335912,0.098904,SDSSJ0011+1446,0.0527643064985,4.9736,5,2,640,860,4.97,0.005,0,8.6155,0.208254,0,4.970,0.005,GGG,2000.0,QSO,3244,SDSSJ0011+1446_B600.fits.gz,Gemini-North,GMOS-N,2011-05-25,1583,4911.0,7831.1,940.0,0,B600
010619.24+004823.3,16.5802,0.8065,4.4488,24.045,0.832,21.113,0.043,19.029,0.024,18.607,0.03,18.313,0.028,0.133,20.521,0.0,0.0,0.0,-9.0,0.0,0.0,17.526,0.327,16.531,0.252,16.038,0.267,0.0,26,-28.572,-0.256,0,1,1,0,8404993,0,1,0,0,1,1,0,2738,52234,51816,396,454,40,5,29,243,8404997,1,1,0,0,1,1,0,24.0,0.807,21.164,0.037,19.055,0.01,18.701,0.013,18.44,0.034,587731513678561523,111686640707043328,SDSS J010619.24+004823.3,2.896888,2.057134,0.404843,0.278306,2.800888,2.082134,0.336843,0.245306,SDSSJ0106+0048,0.0258001939864,4.4502,5,-2,580,800,4.449,0.005,0,6.88373,0.524438,0,4.449,0.005,GGG,2000.0,QSO,17656,SDSSJ0106+0048_B600.fits.gz,Gemini-South,GMOS-S,2011-04-29,1581,4336.3,7243.6,940.0,2,B600
021043.16-001818.4,32.6799,-0.3051,4.7331,24.699,0.964,22.666,0.107,20.505,0.033,19.174,0.027,19.174,0.047,0.147,20.405,14.314,49.137,0.373,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-28.142,-0.433,0,1,1,0,33554433,0,1,0,0,0,0,0,3325,52522,51816,405,263,41,3,326,333,33554433,0,1,0,0,0,0,0,23.914,1.036,22.885,0.16,20.745,0.041,19.278,0.015,19.313,0.06,588015508746797389,114219914696327168,SDSS J0210-0018,1.994192,2.131306,1.312037,-0.017346,0.990192,2.110306,1.448037,-0.052346,SDSSJ0210-0018,0.0285160038797,4.7079,5,-2,610,830,4.77,0.02,0,2.8,0.28,0,4.770,0.02,GGG,2000.0,QSO,32720,SDSSJ0210-0018_B600.fits.gz,Gemini-South,GMOS-S,2011-04-29,1587,4627.1,7552.9,940.0,4,B600
023137.65-072854.4,37.9069,-7.4818,5.4135,24.367,1.206,25.33,0.629,21.565,0.079,19.554,0.027,19.212,0.066,0.169,20.519,0.0,0.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-28.052,0.744,0,1,1,0,1,0,1,0,0,0,0,0,1729,51813,51908,454,610,40,6,333,251,1,0,1,0,0,0,0,0,24.545,1.199,25.356,0.629,21.545,0.078

## Query meta with Query dict

### A simple example

In [4]:
qdict = {'TELESCOPE': 'Gemini-North', 'NPIX': (1580,1583), 'DISPERSER': ['B600', 'R400']}

In [5]:
qmeta = sdb.query_meta(qdict)
qmeta

SDSSJ,RA_GROUP,DEC_GROUP,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAG,JMAGERR,HMAG,HMAGERR,KMAG,KMAGERR,MASSDEL,MASSFLG,MIMAG,DGMI,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,PLATE,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag,zem_GROUP,sig_zem,flag_zem,EPOCH,STYPE,IGM_ID,SPEC_FILE,TELESCOPE,INSTR,DATE-OBS,NPIX,WV_MIN,WV_MAX,R,GROUP_ID,DISPERSER,GROUP
str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str31,str31,str31,float64,float64,float64,float64,float64,float64,float64,float64,str14,float64,float32,int32,int32,int32,int32,float32,float32,int32,float32,float32,int32,float32,float32,str3,float64,str3,int64,str27,str12,str6,str10,int64,float64,float64,float64,int64,str4,str3
001115.23+144601.8,2.8135,14.7672,4.9672,24.087,0.827,22.081,0.074,19.617,0.021,18.277,0.014,18.101,0.025,0.272,20.643,-1.0,0.0,0.0,-9.0,0.0,0.0,17.547,0.342,16.76,0.257,15.796,0.244,0.0,26,-29.191,-0.922,0,1,1,1,8388609,0,1,0,0,1,0,0,2566,52170,52251,751,39,40,3,327,266,8388609,0,1,0,0,1,0,0,24.385,0.941,22.071,0.073,19.642,0.018,18.271,0.013,18.14,0.023,587730773889974538,211612124009463808,SDSS J001115.23+144601.8,1.934192,2.409056,1.304912,0.143904,2.242192,2.374056,1.335912,0.098904,SDSSJ0011+1446,0.0527643064985,4.9736,5,2,640,860,4.97,0.005,0,8.6155,0.208254,0,4.970,0.005,GGG,2000.0,QSO,3244,SDSSJ0011+1446_B600.fits.gz,Gemini-North,GMOS-N,2011-05-25,1583,4911.0,7831.1,940.0,0,B600,GGG
073103.12+445949.4,112.7630,44.9971,5.004,25.465,0.74,25.454,0.472,20.877,0.035,19.226,0.02,19.198,0.055,0.406,20.867,0.0,0.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-28.312,1.351,0,1,1,1,1,0,1,0,0,0,0,0,4264,52963,53314,1866,168,40,4,57,592,1,0,1,0,0,0,0,0,25.467,0.741,25.458,0.473,20.874,0.035,19.214,0.02,19.187,0.055,587738067263619664,525461289133146112,SDSS J073103.12+445949.4,-0.096184,4.494988,1.598626,-0.019908,-0.098184,4.501988,1.607626,-0.020908,SDSSJ0731+4459,0.0787584869059,5.0086,5,2,640,860,4.998,0.005,0,3.22344,0.118597,0,4.998,0.005,GGG,2000.0,QSO,41217,SDSSJ0731+4459_B600.fits.gz,Gemini-North,GMOS-N,2011-05-25,1582,4914.1,7831.9,940.0,8,B600,GGG


### Another example

In [6]:
qdict = {'R': (4000.,1e9), 'WV_MIN': (0., 4000.)}
qmeta = sdb.query_meta(qdict)
qmeta

Name,QSO,RA_GROUP,DEC_GROUP,zem_GROUP,Z_LLS,logNHI,sig(logNHI) [2],SPEC_FILE,sig_zem,flag_zem,STYPE,IGM_ID,EPOCH,NPIX,DATE-OBS,WV_MIN,WV_MAX,R,GROUP_ID,DISPERSER,INSTR,TELESCOPE,GROUP
str33,str19,float64,float64,float64,float64,float64,float64,str37,float64,str4,str3,int64,float64,int64,str10,float64,float64,float64,int64,str4,str5,str13,str10
HD-LLS_J010619.24+004823.3_z4.172,SDSS0106+0048,16.5802,0.8065,4.433,4.17157,19.05,0.2 .. 0.2,HD-LLS_J010619.24+004823.3_MIKE.fits,0.0,UNKN,QSO,17656,2000.0,70180,2003-08-26,3456.4,8108.2,25000.0,5,BOTH,MIKE,Magellan,HD-LLS_DR1
HD-LLS_J012156.03+144823.8_z2.662,SDSS0121+1448,20.4835,14.8066,2.870,2.66231,19.25,0.2 .. 0.15,HD-LLS_J012156.03+144823.8_HIRES.fits,0.0,UNKN,QSO,21499,2000.0,147082,2004-09-08,3243.9,6138.4,48000.0,6,BOTH,HIRES,Keck-I,HD-LLS_DR1
HD-LLS_J012403.80+004432.7_z3.078,SDSS0124+0044,21.0158,0.7424,3.834,3.0776,20.2,0.2 .. 0.2,HD-LLS_J012403.80+004432.7_MIKE.fits,0.0,UNKN,QSO,22001,2000.0,74362,2003-08-28,3314.9,8108.7,25000.0,7,BOTH,MIKE,Magellan,HD-LLS_DR1
HD-LLS_J012700.70-004559.0_z2.944,SDSS0127-00,21.7529,-0.7664,4.066,2.94398,19.8,0.2 .. 0.2,HD-LLS_J012700.70-004559.0_ESI.fits,0.0,UNKN,QSO,22730,2000.0,28124,2002-11-30,3995.9,10203.0,6000.0,8,ECH,ESI,Keck-II,HD-LLS_DR1
HD-LLS_J013340.40+040059.0_z3.995,PSS0133+0400,23.4183,4.0164,4.149,3.99501,20.1,0.3 .. 0.2,HD-LLS_J013340.40+040059.0_ESI.fits,0.0,UNKN,QSO,24295,2000.0,28117,2002-12-01,3994.9,10198.1,6000.0,9,ECH,ESI,Keck-II,HD-LLS_DR1


### One more

In [7]:
qdict = {'R': (1800.,2500), 'WV_MIN': (0., 4000.)}
qmeta = sdb.query_meta(qdict)
qmeta['GROUP'].data

masked_BaseColumn(data = ['BOSS_DR12' 'BOSS_DR12' 'BOSS_DR12' 'BOSS_DR12' 'BOSS_DR12' 'SDSS_DR7'
 'SDSS_DR7' 'SDSS_DR7' 'SDSS_DR7' 'SDSS_DR7'],
                  mask = [False False False False False False False False False False],
            fill_value = N/A)

----

## Query meta at position
    As with query catalog, the position coordinates can have a range of formats

### One simple source

In [8]:
meta = sdb.meta_from_position((0.0019,17.7737), 1*u.arcsec)
meta

Your search yielded 1 match[es] within radius=1 arcsec


SDSS_NAME,RA_GROUP,DEC_GROUP,THING_ID,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,PLATE_DUPLICATE [32],MJD_DUPLICATE [32],FIBERID_DUPLICATE [32],SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_DUPLICATE [32],SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,RUN_NUMBER,PHOTO_MJD,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DATE-OBS,R,zem_GROUP,sig_zem,flag_zem,STYPE,IGM_ID,SPEC_FILE,NPIX,WV_MIN,WV_MAX,GROUP_ID,EPOCH,CAT,INSTR,DISPERSER,TELESCOPE,GROUP
str18,float64,float64,int32,int32,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,int32,int64,int64,int64,int64,int32,int32,int32,int32,int32,int32,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,int32,float64,float64,float64,float64,float64,int16,int32,str3,int16,int16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int16,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,int16,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float64,str10,float64,float64,float64,str9,str3,int64,str28,int64,float64,float64,int64,float64,str5,str4,str4,str10,str9
000000.45+174625.4,0.0019,17.7737,268514930,6173,56238,528,2.30909729004,2.30909729004,0.000943339022342,0,2.30763868197,-1.0,0.346182279084,2.31230791362,2.30638494927,2.3046866469,0,2199023796224,0,0,0,0,-1 .. -1,-1 .. -1,-1 .. -1,0,-1,-1,-1,0,0.537336213179,0.779484925454,-1.0 .. -1.0,0.58568293642,0.915501904005,-1.0,4278.7900341,1595.2886496,2683.5013845,2.15328741853,44.0028643812,1.40097166147,3575.00108901,1554.12165803,2020.87943097,0.706666936497,38.4285671721,15.8517240349,7996.40388327,4579.19012749,3417.21375578,0.223279535625,69.3860000703,17.8808108042,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,7713,54741,301,4,231,1237678601842131080,1.03847694397 .. 2.65843176842,12.3596630096 .. 0.852828204632,22.4397926331 .. 21.3626842499,0.28713452816 .. 0.386405169964,1.03847694397 .. 2.65843176842,-23.882931269,-0.0820910409093,0.146194890141 .. 0.0419441759586,0.113614 .. 0.0338511,21.9297126138,0,0.0,0.0,0.0,

### Multiple meta entries (GGG)

In [9]:
meta = sdb.meta_from_position('001115.23+144601.8', 1*u.arcsec)
meta['WV_MIN'].data

Your search yielded 1 match[es] within radius=1 arcsec


array([ 4911.02528934,  6496.10029461])

### Multiple sources

In [10]:
meta = sdb.meta_from_position((2.813500,14.767200), 20*u.deg)
meta[0:3]

Your search yielded 12 match[es] within radius=20 deg


SDSS_NAME,RA_GROUP,DEC_GROUP,THING_ID,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,PLATE_DUPLICATE [32],MJD_DUPLICATE [32],FIBERID_DUPLICATE [32],SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_DUPLICATE [32],SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,RUN_NUMBER,PHOTO_MJD,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DATE-OBS,R,zem_GROUP,sig_zem,flag_zem,STYPE,IGM_ID,SPEC_FILE,NPIX,WV_MIN,WV_MAX,GROUP_ID,EPOCH,CAT,INSTR,DISPERSER,TELESCOPE,GROUP,Name,QSO,Z_LLS,logNHI,sig(logNHI) [2],SDSSJ,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAGERR,HMAGERR,KMAGERR,MASSDEL,MASSFLG,MIMAG,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag
str18,float64,float64,int32,int32,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,int32,int64,int64,int64,int64,int32,int32,int32,int32,int32,int32,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,int32,float64,float64,float64,float64,float64,int16,int32,str3,int16,int16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int16,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,int16,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float64,str10,float64,float64,float64,str10,str3,int64,str37,int64,float64,float64,int64,float64,str5,str6,str4,str13,str10,str33,str19,float64,float64,float64,str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64

In [11]:
meta['GROUP'].data

masked_BaseColumn(data = ['BOSS_DR12' 'BOSS_DR12' 'BOSS_DR12' 'BOSS_DR12' 'BOSS_DR12' 'HD-LLS_DR1'
 'HD-LLS_DR1' 'GGG' 'GGG' 'GGG' 'GGG' 'SDSS_DR7' 'SDSS_DR7' 'SDSS_DR7'
 'SDSS_DR7'],
                  mask = [False False False False False False False False False False False False
 False False False],
            fill_value = N/A)

#### Restrict on groups

In [12]:
meta = sdb.meta_from_position((2.813500,14.767200), 20*u.deg, groups=['GGG','HD-LLS_DR1'])

Your search yielded 3 match[es] within radius=20 deg


In [13]:
meta['GROUP'].data

masked_BaseColumn(data = ['HD-LLS_DR1' 'HD-LLS_DR1' 'GGG' 'GGG' 'GGG' 'GGG'],
                  mask = [False False False False False False],
            fill_value = N/A)

----

## Query Meta with Coordinates list
    When querying with a coordinate list, there are two approaches to 
    the data returned.  The default is to return the meta data
    for the first spectrum matched to each coordinate.
    
    The other option is to retrieve all of the meta data for each 
    coordinate input.  The returned object is then a list of bool arrays
    and a Table of all the meta data.
    
    We provide examples for each.

### Meta for first match for each coordinate
    Returns a Table with a single meta data entry per coordinate even if multiple exist.
    If there is no match, the row is empty in the Table.

    If there are zero matches, return None.

#### Single source (which matches)

In [14]:
coord = SkyCoord(ra=0.0019, dec=17.7737, unit='deg')
matches, meta = sdb.meta_from_coords(coord)
meta

Your search yielded 1 matches from 1 input coordinates
Final query yielded 1 matches.


SDSS_NAME,RA_GROUP,DEC_GROUP,THING_ID,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,PLATE_DUPLICATE [32],MJD_DUPLICATE [32],FIBERID_DUPLICATE [32],SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_DUPLICATE [32],SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,RUN_NUMBER,PHOTO_MJD,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DATE-OBS,R,zem_GROUP,sig_zem,flag_zem,STYPE,IGM_ID,SPEC_FILE,NPIX,WV_MIN,WV_MAX,GROUP_ID,EPOCH,CAT,INSTR,DISPERSER,TELESCOPE,GROUP
str18,float64,float64,int32,int32,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,int32,int64,int64,int64,int64,int32,int32,int32,int32,int32,int32,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,int32,float64,float64,float64,float64,float64,int16,int32,str3,int16,int16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int16,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,int16,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float64,str10,float64,float64,float64,str9,str3,int64,str28,int64,float64,float64,int64,float64,str5,str4,str4,str10,str9
000000.45+174625.4,0.00189828518376,17.7737391299,268514930,6173,56238,528,2.30909729004,2.30909729004,0.000943339022342,0,2.30763868197,-1.0,0.346182279084,2.31230791362,2.30638494927,2.3046866469,0,2199023796224,0,0,0,0,-1 .. -1,-1 .. -1,-1 .. -1,0,-1,-1,-1,0,0.537336213179,0.779484925454,-1.0 .. -1.0,0.58568293642,0.915501904005,-1.0,4278.7900341,1595.2886496,2683.5013845,2.15328741853,44.0028643812,1.40097166147,3575.00108901,1554.12165803,2020.87943097,0.706666936497,38.4285671721,15.8517240349,7996.40388327,4579.19012749,3417.21375578,0.223279535625,69.3860000703,17.8808108042,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,7713,54741,301,4,231,1237678601842131080,1.03847694397 .. 2.65843176842,12.3596630096 .. 0.852828204632,22.4397926331 .. 21.3626842499,0.28713452816 .. 0.386405169964,1.03847694397 .. 2.65843176842,-23.882931269,-0.0820910409093,0.146194890141 .. 0.0419441759586,0.113614 .. 0.0338511,21.929712613

#### Single source which fails to match

In [15]:
coord = SkyCoord(ra=0.0019, dec=-17.7737, unit='deg')
matches, meta = sdb.meta_from_coords(coord)
print(meta)

Your search yielded 0 matches from 1 input coordinates
None


#### Source where multiple spectra exist, but only the first record is returned

In [16]:
coord = SkyCoord(ra=2.813458, dec=14.767167, unit='deg')
_, meta = sdb.meta_from_coords(coord)
meta

Your search yielded 1 matches from 1 input coordinates
Final query yielded 1 matches.


SDSSJ,RA_GROUP,DEC_GROUP,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAG,JMAGERR,HMAG,HMAGERR,KMAG,KMAGERR,MASSDEL,MASSFLG,MIMAG,DGMI,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,PLATE,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag,zem_GROUP,sig_zem,flag_zem,EPOCH,STYPE,IGM_ID,SPEC_FILE,TELESCOPE,INSTR,DATE-OBS,NPIX,WV_MIN,WV_MAX,R,GROUP_ID,DISPERSER,GROUP
str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str31,str31,str31,float64,float64,float64,float64,float64,float64,float64,float64,str14,float64,float32,int32,int32,int32,int32,float32,float32,int32,float32,float32,int32,float32,float32,str3,float64,str3,int64,str27,str12,str6,str10,int64,float64,float64,float64,int64,str4,str3
001115.23+144601.8,2.813485,14.767169,4.9672,24.087,0.827,22.081,0.074,19.617,0.021,18.277,0.014,18.101,0.025,0.272,20.643,-1.0,0.0,0.0,-9.0,0.0,0.0,17.547,0.342,16.76,0.257,15.796,0.244,0.0,26,-29.191,-0.922,0,1,1,1,8388609,0,1,0,0,1,0,0,2566,52170,52251,751,39,40,3,327,266,8388609,0,1,0,0,1,0,0,24.385,0.941,22.071,0.073,19.642,0.018,18.271,0.013,18.14,0.023,587730773889974538,211612124009463808,SDSS J001115.23+144601.8,1.934192,2.409056,1.304912,0.143904,2.242192,2.374056,1.335912,0.098904,SDSSJ0011+1446,0.0527643064985,4.9736,5,2,640,860,4.97,0.005,0,8.6155,0.208254,0,4.97,0.005,GGG,2000.0,QSO,3244,SDSSJ0011+1446_B600.fits.gz,Gemini-North,GMOS-N,2011-05-25,1583,4911.02528934,7831.05124185,940.0,0,B600,GGG


#### Multiple coordinates, each matched

In [17]:
coords = SkyCoord(ra=[0.0028,2.813458], dec=[14.9747,14.767167], unit='deg')
matches, meta = sdb.meta_from_coords(coords)
print(matches)
meta

Your search yielded 2 matches from 2 input coordinates
Final query yielded 2 matches.
[ True  True]


SDSS_NAME,RA_GROUP,DEC_GROUP,THING_ID,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,PLATE_DUPLICATE [32],MJD_DUPLICATE [32],FIBERID_DUPLICATE [32],SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_DUPLICATE [32],SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,RUN_NUMBER,PHOTO_MJD,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DATE-OBS,R,zem_GROUP,sig_zem,flag_zem,STYPE,IGM_ID,SPEC_FILE,NPIX,WV_MIN,WV_MAX,GROUP_ID,EPOCH,CAT,INSTR,DISPERSER,TELESCOPE,GROUP,SDSSJ,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAGERR,HMAGERR,KMAGERR,MASSDEL,MASSFLG,MIMAG,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag
str18,float64,float64,int32,int32,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,int32,int64,int64,int64,int64,int32,int32,int32,int32,int32,int32,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,int32,float64,float64,float64,float64,float64,int16,int32,str3,int16,int16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int16,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,int16,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float64,str10,float64,float64,float64,str9,str3,int64,str28,int64,float64,float64,int64,float64,str5,str6,str4,str12,str9,str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,i

#### Multiple coordinates, one fails to match by coordinate

In [18]:
coords = SkyCoord(ra=[0.0028,9.99,2.813458], dec=[14.9747,-9.99,14.767167], unit='deg')
matches, meta = sdb.meta_from_coords(coords)
print(matches)
meta

Your search yielded 2 matches from 3 input coordinates
Final query yielded 2 matches.
[ True False  True]


SDSS_NAME,RA_GROUP,DEC_GROUP,THING_ID,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,PLATE_DUPLICATE [32],MJD_DUPLICATE [32],FIBERID_DUPLICATE [32],SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_DUPLICATE [32],SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,RUN_NUMBER,PHOTO_MJD,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DATE-OBS,R,zem_GROUP,sig_zem,flag_zem,STYPE,IGM_ID,SPEC_FILE,NPIX,WV_MIN,WV_MAX,GROUP_ID,EPOCH,CAT,INSTR,DISPERSER,TELESCOPE,GROUP,SDSSJ,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAGERR,HMAGERR,KMAGERR,MASSDEL,MASSFLG,MIMAG,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag
str18,float64,float64,int32,int32,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,int32,int64,int64,int64,int64,int32,int32,int32,int32,int32,int32,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,int32,float64,float64,float64,float64,float64,int16,int32,str3,int16,int16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int16,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,int16,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float64,str10,float64,float64,float64,str9,str3,int64,str28,int64,float64,float64,int64,float64,str5,str6,str4,str12,str9,str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,i

#### Multiple coordiantes, one fails to match input group list

In [19]:
coords = SkyCoord(ra=[0.0028,2.813458], dec=[14.9747,14.767167], unit='deg')
matches, meta = sdb.meta_from_coords(coords, groups=['GGG'])
print(matches)
print(meta['IGM_ID'])
meta

Your search yielded 1 matches from 2 input coordinates
Final query yielded 1 matches.
[False  True]
IGM_ID
------
    -2
  3244


SDSSJ,RA_GROUP,DEC_GROUP,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAG,JMAGERR,HMAG,HMAGERR,KMAG,KMAGERR,MASSDEL,MASSFLG,MIMAG,DGMI,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,PLATE,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag,zem_GROUP,sig_zem,flag_zem,EPOCH,STYPE,IGM_ID,SPEC_FILE,TELESCOPE,INSTR,DATE-OBS,NPIX,WV_MIN,WV_MAX,R,GROUP_ID,DISPERSER,GROUP
str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str31,str31,str31,float64,float64,float64,float64,float64,float64,float64,float64,str14,float64,float32,int32,int32,int32,int32,float32,float32,int32,float32,float32,int32,float32,float32,str3,float64,str3,int64,str27,str12,str6,str10,int64,float64,float64,float64,int64,str4,str3
--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,-2,--,--,--,--,--,--,--,--,--,--,--
001115.23+144601.8,2.813485,14.767169,4.9672,24.087,0.827,22.081,0.074,19.617,0.021,18.277,0.014,18.101,0.025,0.272,20.643,-1.0,0.0,0.0,-9.0,0.0,0.0,17.547,0.342,16.76,0.257,15.796,0.244,0.0,26,-29.191,-0.922,0,1,1,1,8388609,0,1,0,0,1,0,0,2566,52170,52251,751,39,40,3,327,266,8388609,0,1,0,0,1,0,0,24.385,0.941,22.071,0.073,19.642,0.018,18.271,0.013,18.14,0.023,587730773889974538,211612124009463808,SDSS J001115.23+144601.8,1.934192,2.409056,1.304912,0.143904,2.242192,2.374056,1.335912,0.098904,SDSSJ0011+1446,0.0527643064985,4.9736,5,2,640,860,4.97,0.005,0,8.6155,0.208254,0,4.97,0.005,GGG,2000.0,QSO,3244,SDSSJ0011+1446_B600.fits.gz,Gemini-North,GMOS-N,2011-05-25,1583,4911.02528934,7831.05124185,940.0,0,B600,GGG


### All Meta Data for each input coordinate
    Here, a list of bool arrays relative to stacked meta table is returned.
    This is a bit convoluted, but is *much* faster for large coordinate lists.
    If there is no match to a given coordinate, the entry in the list is None

#### Two sources.  The second one has two spectra in the database

In [21]:
coords = SkyCoord(ra=[0.0028,2.813458], dec=[14.9747,14.767167], unit='deg')
matches, list_of_meta, meta_stack = sdb.meta_from_coords(coords, first=False)
print('Matches = ', matches)
list_of_meta, meta_stack[list_of_meta[0]]

Your search yielded 2 matches from 2 input coordinates
('Matches = ', array([ True,  True], dtype=bool))


([masked_array(data = [True False False],
               mask = [False False False],
         fill_value = True), masked_array(data = [False True True],
               mask = [False False False],
         fill_value = True)], <Table masked=True length=1>
     SDSS_NAME      RA_GROUP DEC_GROUP ... flux1450 fluxerror1450 skyflag
       str18        float64   float64  ... float32     float32     int32 
 ------------------ -------- --------- ... -------- ------------- -------
 000000.66+145828.8   0.0028   14.9747 ...       --            --      --)

#### Two sources, limit by groups

In [22]:
matches, list_of_meta, meta_stack = sdb.meta_from_coords(coords, first=False, groups=['GGG'])
list_of_meta, meta_stack[list_of_meta[1]]

Your search yielded 1 matches from 2 input coordinates


([None, array([ True,  True], dtype=bool)], <Table length=2>
       SDSSJ        RA_GROUP DEC_GROUP  z_s10  ... GROUP_ID DISPERSER GROUP
       str31        float64   float64  float64 ...  int64      str4    str3
 ------------------ -------- --------- ------- ... -------- --------- -----
 001115.23+144601.8   2.8135   14.7672  4.9672 ...        0      B600   GGG
 001115.23+144601.8   2.8135   14.7672  4.9672 ...      163      R400   GGG)

#### Three sources; second one has no match

In [23]:
coords = SkyCoord(ra=[0.0028,9.99,2.813458], dec=[14.9747,-9.99,14.767167], unit='deg')
matches, list_of_meta, meta_stack = sdb.meta_from_coords(coords, first=False)
print('Matches = ', matches)
meta_stack[list_of_meta[0]]

Your search yielded 2 matches from 3 input coordinates
('Matches = ', array([ True, False,  True], dtype=bool))


SDSS_NAME,RA_GROUP,DEC_GROUP,THING_ID,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,PLATE_DUPLICATE [32],MJD_DUPLICATE [32],FIBERID_DUPLICATE [32],SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_DUPLICATE [32],SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,RUN_NUMBER,PHOTO_MJD,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,DATE-OBS,R,zem_GROUP,sig_zem,flag_zem,STYPE,IGM_ID,SPEC_FILE,NPIX,WV_MIN,WV_MAX,GROUP_ID,EPOCH,CAT,INSTR,DISPERSER,TELESCOPE,GROUP,SDSSJ,z_s10,UMAG,UMAGERR,GMAG,GMAGERR,RMAG,RMAGERR,IMAG,IMAGERR,ZMAG,ZMAGERR,AU,logNH,FIRSTMAG,FIRSTSN,FIRSTDEL,RASSCNT,RASSSN,RASSDEL,JMAGERR,HMAGERR,KMAGERR,MASSDEL,MASSFLG,MIMAG,MFLAG,SPFLAG,MODEFLAG,USELFLAG,BESTFLAG,LZFLAG,HZFLAG,FTFLAG,RTFLAG,SRFLAG,STFLAG,GXFLAG,RUN,RMJD,SMJD,FIBER,RERUN,CAMCOL,FIELD,OBJECT,TFLAG,LZTFLAG,HZTFLAG,FTTFLAG,RTTFLAG,SRTFLAG,STTFLAG,GXTFLAG,UTMAG,UTMAGERR,GTMAG,GTMAGERR,RTMAG,RTMAGERR,ITMAG,ITMAGERR,ZTMAG,ZTMAGERR,BESTID,SPECOID,ONAME,u_minus_g,g_minus_r,r_minus_i,i_minus_z,t_u_minus_g,t_g_minus_r,t_r_minus_i,t_i_minus_z,name,ebv,z_hw10,method_hw10,telsemester,cw_b600,cw_r400,z_gmos,zerror_gmos,absflag,flux1450,fluxerror1450,skyflag
str18,float64,float64,int32,int32,int32,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,int32,int64,int64,int64,int64,int32,int32,int32,int32,int32,int32,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,float64,float64,int32,float64,float64,float64,float64,float64,int16,int32,str3,int16,int16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int16,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,int16,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float64,str10,float64,float64,float64,str9,str3,int64,str28,int64,float64,float64,int64,float64,str5,str6,str4,str12,str9,str31,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,i